In [51]:
import pandas as pd

df = pd.read_csv('nfl-cleaned.csv')

In [52]:
df.sort_values('Date')

df = df.reset_index()

We must now compute the expanding averages over previous games in order to train
and test our model. $\newline$
This is because we can only use statistics from previous 
games to predict our incoming test game. $\newline$
We use expanding instead of rolling averages because we want the expanding mean
over all previous games, not just a fixed sliding window.

In [53]:
df[df['Team1'] == 'DAL'].head()

,index,Unnamed: 0,ID,Team1Won,Season,Date,Team1,Team2,Home,Team1Pts,...,Team1RushAtt,Team2RushAtt,Team1RushYds,Team2RushYds,Team1RYM,Team2RYM,Team1PYM,Team2PYM,Team1YM,Team2YM
6,6,6,1459,False,2022,2022-09-11,DAL,TAM,1,3,...,18,33,71,152,-81,81,-22,22,-103,103
47,47,47,1383,True,2022,2022-09-26,DAL,NYG,0,23,...,30,25,176,167,9,-9,46,-46,55,-55
53,53,53,1359,True,2022,2022-10-02,DAL,WAS,1,25,...,29,27,62,142,-80,80,62,-62,-18,18
74,74,74,1341,True,2022,2022-10-09,DAL,LAR,0,22,...,34,15,163,38,125,-125,-209,209,-84,84
88,88,88,1309,False,2022,2022-10-16,DAL,PHI,0,17,...,26,39,134,136,-2,2,49,-49,47,-47


In [54]:
# Compute the expanding averages over previous games. 

numeric_columns = ['Home',
				'Team1Pts',    
				'Team2Pts',    
				'Team1PtDiff',
				'Team2PtDiff', 
				'Team1TM',    
				'Team2TM',     
				'Team1Rating', 
				'Team2Rating', 
				'Team1Sks',    
				'Team2Sks',    
				'Team1SkYds',  
				'Team2SkYds',  
				'Team1RushAtt',
				'Team2RushAtt',
				'Team1RushYds',
				'Team2RushYds',
				'Team1RYM',    
				'Team2RYM',    
				'Team1PYM',    
				'Team2PYM',    
				'Team1YM',     
				'Team2YM']

for column in numeric_columns:
	avg_col_name = column + '_avg'
	df[avg_col_name] = (
		df.groupby('Team1', group_keys=False)[column]
		.apply(lambda group: group.expanding().mean().shift(1))
		.reset_index(drop=True)
	)

In [55]:
df[df['Team1'] == 'DAL'].head()

,index,Unnamed: 0,ID,Team1Won,Season,Date,Team1,Team2,Home,Team1Pts,...,Team1RushAtt_avg,Team2RushAtt_avg,Team1RushYds_avg,Team2RushYds_avg,Team1RYM_avg,Team2RYM_avg,Team1PYM_avg,Team2PYM_avg,Team1YM_avg,Team2YM_avg
6,6,6,1459,False,2022,2022-09-11,DAL,TAM,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,47,47,1383,True,2022,2022-09-26,DAL,NYG,0,23,...,18.000000,33.000000,71.0,152.000000,-81.000000,81.000000,-22.000000,22.000000,-103.0,103.0
53,53,53,1359,True,2022,2022-10-02,DAL,WAS,1,25,...,24.000000,29.000000,123.5,159.500000,-36.000000,36.000000,12.000000,-12.000000,-24.0,24.0
74,74,74,1341,True,2022,2022-10-09,DAL,LAR,0,22,...,25.666667,28.333333,103.0,153.666667,-50.666667,50.666667,28.666667,-28.666667,-22.0,22.0
88,88,88,1309,False,2022,2022-10-16,DAL,PHI,0,17,...,27.750000,25.000000,118.0,124.750000,-6.750000,6.750000,-30.750000,30.750000,-37.5,37.5


In [56]:
# TODO: Should we drop the first column or impute with its original values?

for column in numeric_columns:
	avg_col_name = column + '_avg'
	df[avg_col_name] = df[avg_col_name].fillna(df[column])

df[df['Team1'] == 'DAL'].head()

,index,Unnamed: 0,ID,Team1Won,Season,Date,Team1,Team2,Home,Team1Pts,...,Team1RushAtt_avg,Team2RushAtt_avg,Team1RushYds_avg,Team2RushYds_avg,Team1RYM_avg,Team2RYM_avg,Team1PYM_avg,Team2PYM_avg,Team1YM_avg,Team2YM_avg
6,6,6,1459,False,2022,2022-09-11,DAL,TAM,1,3,...,18.000000,33.000000,71.0,152.000000,-81.000000,81.000000,-22.000000,22.000000,-103.0,103.0
47,47,47,1383,True,2022,2022-09-26,DAL,NYG,0,23,...,18.000000,33.000000,71.0,152.000000,-81.000000,81.000000,-22.000000,22.000000,-103.0,103.0
53,53,53,1359,True,2022,2022-10-02,DAL,WAS,1,25,...,24.000000,29.000000,123.5,159.500000,-36.000000,36.000000,12.000000,-12.000000,-24.0,24.0
74,74,74,1341,True,2022,2022-10-09,DAL,LAR,0,22,...,25.666667,28.333333,103.0,153.666667,-50.666667,50.666667,28.666667,-28.666667,-22.0,22.0
88,88,88,1309,False,2022,2022-10-16,DAL,PHI,0,17,...,27.750000,25.000000,118.0,124.750000,-6.750000,6.750000,-30.750000,30.750000,-37.5,37.5


Let's hold out a test set and set it aside for later use. $\newline$
Note that we can simply split the dataframe because it has already been sorted in chronological order, meaning that data leakage for the time series logic will **not** occur.

In [57]:
import numpy as np

train_set, test_set = np.split(df, [int(0.8 * len(df))])

Let's do cross validation and training!

The function below sets up training and testing data for each fold of a time series split. It begins by defining column groups: `train_post_game_cols` holds post-game statistics like points and rushing yards, while `test_post_game_col` contains the same stats but with `_avg` appended, representing averages for testing. The function takes a specific fold (training and testing indices) and a dataframe as input, then uses these indices to split the dataframe into training and testing sets. Features and outcome labels are extracted separately for training (`X_train` and `y_train`) and testing (`X_test` and `y_test`). The function ensures the split data is ready for training and evaluation during the cross-validation process.

In [58]:
# Prepare train and test sets for each fold of TimeSeriesSplit

pre_game_cols = ['Team1', 'Team2', 'Home']
train_post_game_cols = ['Team1Pts', 'Team2Pts', 'Team1RushYds', 'Team2RushYds', 'Team1SkYds', 'Team2SkYds',
                  'Team1Sks', 'Team2Sks', 'Team1RushAtt', 'Team2RushAtt', 'Team1RYM', 'Team2RYM', 
                  'Team1PYM', 'Team2PYM', 'Team1YM', 'Team2YM', 'Team1Rating', 'Team2Rating']

test_post_game_cols = [col + '_avg' for col in train_post_game_cols]

outcome_col = 'Team1Won'

def prep_data_for_fold(fold, df):
  
  # Split data into training and testing based on the fold
  train_indices, test_indices = fold
  train_data = df.iloc[train_indices]
  test_data = df.iloc[test_indices]

  # Extract features that will be trained and tested on
  X_train = train_data[train_post_game_cols]
  X_test = test_data[test_post_game_cols]
  
  # Class labels from fold split
  y_train = train_data[outcome_col]
  y_test = test_data[outcome_col]
  
  return X_train, X_test, y_train, y_test

The function below calculates how well a model performs on the NFL time series dataset using a nested cross-validation setup. It uses a `TimeSeriesSplit` with 5 splits to preserve the order of time-dependent data. For each fold, it splits the data into training and testing sets and prepares the features and labels using `prep_data_for_fold`. Columns in the test set ending with `_avg` are renamed to match their training counterparts. The function scales the data by fitting a `StandardScaler` on the training set and applying it to both training and testing data. Then, it runs a `GridSearchCV` on the training set to find the best hyperparameters for the model. The best-performing model from the grid search is used to make predictions on the test set, and its accuracy is calculated and stored. After completing all folds, the function returns the average accuracy and prints the best model configuration.

In [59]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# TODO: PCA???
# TODO: Run SequentialFeatureSelector to only use best features?

# Nested cross-validation loop that computes how well a model does.
# Return an accuracy score averaged over all folds of the TimeSeriesSplit.

best_models = []

def get_model_accuracy(model, params, df):

  tscv = TimeSeriesSplit(n_splits=5)

  accuracies = []

  # Outer loop: find average accuracy over all time series splits

  for train_indices, test_indices in tscv.split(df):

    # Prepare the data for this fold
    X_train, X_test, y_train, y_test = prep_data_for_fold((train_indices, test_indices), df)

    # Rename the <col>_avg columns to just <col> so GridSearchCV doesn't complain
    X_test = X_test.rename(columns=lambda x: x[:-4] if x.endswith('_avg') else x)

    # Scale the data (fit scaler on training data and transform both train and test)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Inner loop: find the best hyperparameters for this split
    grid_search = GridSearchCV(estimator=model, param_grid=params, cv=tscv)
    grid_search.fit(X_train, y_train)

    # Get the best model from grid search
    best_model = grid_search.best_estimator_

    # Analyze how well model does by comparing its predictions to actual class labels
    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    
  best_models.append(best_model)
  
  print(best_model)

  # Return the average accuracy across all outer folds
  return sum(accuracies) / len(accuracies)

In [60]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import warnings

warnings.filterwarnings("ignore")

models_dict = {}

dtc = DecisionTreeClassifier()
dtc_params = {
  'max_depth': [5, 10, 15, 20],
  'max_features': [5, 10, 15],
  'min_samples_leaf': [5, 10, 15, 20]
}

lr = LogisticRegression()
lr_params = {
	'penalty': ['l1', 'l2', 'elasticnet', None],
	'C': [0.1, 1, 10],
	'solver': ['liblinear', 'saga'],
	'max_iter': [100, 200]
}

rfc = RandomForestClassifier()
rfc_params = {
  'n_estimators': [50, 100, 200],
  'max_depth': [None, 10, 20],
  'min_samples_split': [2, 5, 10],
  'min_samples_leaf': [1, 2, 4]
}

gbc = GradientBoostingClassifier()
gbc_params = {
  'n_estimators': [50, 100, 200],
  'learning_rate': [0.01, 0.1, 0.2],
  'max_depth': [3, 5, 7],
  'subsample': [0.8, 1.0]
}

knn = KNeighborsClassifier()
knn_params = {
  'n_neighbors': [3, 5, 10],
  'weights': ['uniform', 'distance'],
  'p': [1, 2]  # 1 = Manhattan distance, 2 = Euclidean distance
}

svc = SVC()
svc_params = {
  'C': [0.1, 1, 10],
  'kernel': ['linear', 'rbf', 'poly'],
  'gamma': ['scale', 'auto'],
  'degree': [2, 3, 4]  # Only for 'poly' kernel
}

models_dict[dtc] = dtc_params
models_dict[lr] = lr_params
# models_dict[rfc] = rfc_params
# models_dict[gbc] = gbc_params
models_dict[knn] = knn_params
models_dict[svc] = svc_params

for model in models_dict.keys():
  score = get_model_accuracy(model, models_dict[model], df)
  print(score)

# TODO: Handle <col>_avg NaN for each team's first game of the season
# LogisticRegression() does not accept missing values encoded as NaN natively.

DecisionTreeClassifier(max_depth=15, max_features=15, min_samples_leaf=5)
0.5626016260162602
LogisticRegression(C=0.1, penalty='l1', solver='saga')
0.6097560975609756
KNeighborsClassifier(n_neighbors=10, weights='distance')
0.5772357723577235
SVC(C=1, degree=2, kernel='linear')
0.6032520325203252


Let's do a final evaluation of the model on the held out test set.

In [61]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Let's say that LogisticRegression(C=0.1, penalty='l1', solver='saga') is the best model.

lr = LogisticRegression(C=0.1, solver='saga', penalty='l1')

for model in best_models:
	X_train = train_set[train_post_game_cols]
	X_test = test_set[test_post_game_cols].rename(columns=lambda x: x[:-4] if x.endswith('_avg') else x)
	y_train = train_set[outcome_col]
	y_test = test_set[outcome_col]

	scaler = StandardScaler()
	X_train = scaler.fit_transform(X_train)
	X_test = scaler.transform(X_test)

	model.fit(X_train, y_train)
	y_pred = model.predict(X_test)
	accuracy = accuracy_score(y_test, y_pred)

	print("Accuracy of model " + str(model) + ": " + str(accuracy))

Accuracy of model DecisionTreeClassifier(max_depth=15, max_features=15, min_samples_leaf=5): 0.527027027027027
Accuracy of model LogisticRegression(C=0.1, penalty='l1', solver='saga'): 0.581081081081081
Accuracy of model KNeighborsClassifier(n_neighbors=10, weights='distance'): 0.581081081081081
Accuracy of model SVC(C=1, degree=2, kernel='linear'): 0.5675675675675675


In [62]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_pred)
TP = conf_matrix[0, 0]
FN = conf_matrix[0, 1]
FP = conf_matrix[1, 0]
TN = conf_matrix[1, 1]

In [63]:
def print_confusion_matrix(TP, FN, FP, TN):
    table_data = [[TP,FN],[FP,TN]]
    df = pd.DataFrame(table_data, columns =['Predicted 1','Predicted 0'])
    df = df.rename(index={0: 'Actual 1', 1: 'Actual 0'})
    display(df)

In [64]:
print_confusion_matrix(TP, FN, FP, TN)

,Predicted 1,Predicted 0
Actual 1,41,26
Actual 0,38,43
